In [ ]:
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

golden_mean = (np.sqrt(5) - 1) / 2 # Aesthetic ratio
fig_width_pt = 246.0 # Columnwidth
inches_per_pt = 1.0 / 72.27 # Convert pt to inches
fig_width = fig_width_pt * inches_per_pt
fig_height = fig_width * golden_mean # height in inches
fig_size = [fig_width, fig_height]

params = {'backend': 'ps',
          'axes.labelsize': 10,
          'font.size': 10,
          'legend.fontsize': 8,
          'xtick.labelsize': 8,
          'ytick.labelsize': 8,
          'text.usetex': True,
          'figure.figsize': fig_size,
          'font.family': 'serif',
          'font.serif': 'Computer Modern Roman',
          'legend.frameon': True,
          'savefig.dpi': 300,
          'figure.dpi': 300,
         }

plt.rcParams.update(params)
plt.rc('text.latex', preamble=[r'\usepackage{color}', r'\usepackage{bm}'])

import matplotlib.colors as colors
    
class HistogramNormalize(colors.Normalize):
    def __init__(self, data, vmin=None, vmax=None, mixing_degree=1):
        self.mixing_degree = mixing_degree
        if vmin is not None:
            data = data[data >= vmin]
        if vmax is not None:
            data = data[data <= vmax]

        self.sorted_data = np.sort(data.flatten())
        colors.Normalize.__init__(self, vmin, vmax)

    def __call__(self, value, clip=None):
        hist_norm = np.ma.masked_array(np.searchsorted(self.sorted_data, value) /
                                       len(self.sorted_data))
        linear_norm = super().__call__(value, clip)
        return self.mixing_degree * hist_norm + (1 - self.mixing_degree) * linear_norm

    
def open_mtx(fname):
    with open(fname, 'rb') as f:
        line1 = f.readline()
        *shape, entry_size = [int(i) for i in  f.readline().split()]
        print('shape: ', shape)
        data = np.fromfile(f, (np.float64 if entry_size == 8 else np.float32))
        data = data.reshape(shape).squeeze()
    
    keys = ['units', 'plot_settings', 'xname', 'xmin', 'xmax', 
            'yname', 'ymin', 'ymax', 'zname', 'zmin', 'zmax']
    values = line1.decode("utf-8").replace('\n', '').split(',')
    meta_data = dict(zip(keys, values))
    meta_data = {}
    for key, val in zip(keys, values):
        try:
            meta_data[key] = float(val)
        except:
            meta_data[key] = val
    return meta_data, data

# Fig 1

In [ ]:
from matplotlib._png import read_png
from matplotlib.offsetbox import TextArea, DrawingArea, OffsetImage, AnnotationBbox
meta, dat = open_mtx('data/experimental_data/fig1.mtx')
meta['ymax'], meta['ymin'] = meta['ymin'], meta['ymax'] 
fig = plt.figure(figsize=(fig_width, fig_height))
ax = fig.add_axes([0.1, 0.2, 0.71, 0.85])
extent = (meta['xmin'], meta['xmax'], meta['ymin'], meta['ymax'])
norm = HistogramNormalize(dat, vmin=0, 
                          vmax=16, 
                          mixing_degree=0.5)
kwargs = dict(extent=extent, cmap='viridis', aspect='auto', 
              norm=norm, interpolation='nearest',
              origin='lower', vmax=norm.vmax)
im = ax.imshow(np.rot90(dat), **kwargs)
ax.set_xlabel('$B$ [$T$]')
ax.set_ylabel(r'$I_\textrm{bias}$ [nA]')
cax = fig.add_axes([0.83, 0.2, 0.04, 0.85])
cb = fig.colorbar(im, cax=cax, extend='max', label=r'$\mathrm{d}V/\mathrm{d}I$ [k$\Omega$]')
cbar_ticks = [0, 5, 10, 15]
cb.set_ticks(cbar_ticks)

# imagebox = OffsetImage(read_png('figures/setup.png')[::4,::4], zoom=0.2, dpi_cor=False)

# xy = [1.52, 1.3]
# ab = AnnotationBbox(imagebox, xy,
#                     xycoords='data',
#                     boxcoords="offset points",
#                     pad=0, annotation_clip=False)

# ax.add_artist(ab)

fig.savefig('figures/fig1_2.pdf', format='pdf', dpi=300,
            additional_artists=[ab, imagebox], 
            bbox_inches='tight', 
            pad_inches=0)
plt.show()

In [ ]:
!inkscape -l figures/setup.svg figures/setup.pdf

In [ ]:
!svg_stack.py --direction=v --margin=30 setup.svg fig1_2.svg > fig1.svg


In [ ]:
!inkscape -l figures/fig1.pdf figures/fig1.svg

# Fig 2

In [ ]:
fig, axs = plt.subplots(2, 2, sharey=True, sharex=True, figsize=(fig_width, fig_height*1.3))
plt.subplots_adjust(bottom=0.2, left=0.125, right=0.80, top=0.9, hspace=0.10, wspace=0.10)


exp = [open_mtx('data/experimental_data/fig2_1.mtx'),
       open_mtx('data/experimental_data/fig2_2.mtx'),
       open_mtx('data/experimental_data/fig2_3.mtx'),
       open_mtx('data/experimental_data/fig2_4.mtx')]

for i, ax in enumerate(axs.reshape(-1)):
    meta, dat = exp[i]
    extent = (meta['xmin'], meta['xmax'], meta['ymin'], meta['ymax'])
    print(extent)
    data = dat.T / 1000
    if i == 0:
        norm = HistogramNormalize(data, vmin=0, vmax=22, mixing_degree=0.6)
        kwargs = dict(extent=extent, cmap='viridis', aspect='auto', 
                      norm=norm, interpolation='nearest',
                      origin='lower')
        im = ax.imshow(data, **kwargs)
    else:
        ax.imshow(data, **kwargs)
    ax.set_xticks([0, .2, 0.4])
    ax.set_yticks([0, 1, 2, 3])
    ax.set_ylim(0, 3.6)
    ax.set_xlim(0, 0.45)
    ax.set_xlabel('')
    if i in [0, 2]:
        ax.set_ylabel(r'$I_\textrm{bias}$ [nA]')
    if i in [2, 3]:
        ax.set_xlabel('$B$ [$T$]')
    ax.text(0.88, 0.86, '({})'.format('abcd'[i]),
        horizontalalignment='center', verticalalignment='center',
        transform=ax.transAxes, fontdict={'color': 'w'})

    names = [r'$2.64\;\mathrm{V}$',
             r'$2.66\;\mathrm{V}$',
             r'$2.68\;\mathrm{V}$',
             r'$2.70\;\mathrm{V}$']
    
    ax.text(0.5, 0.84, names[i],
        horizontalalignment='center', verticalalignment='center',
        transform=ax.transAxes, fontdict={'color': 'w'})

cax = fig.add_axes([0.83, 0.2, 0.04, 0.7])
cbar_ticks = [0, 5, 10, 15, 20, 25]
cb = fig.colorbar(im, cax=cax, extend='max', label=r'$\mathrm{d}V/\mathrm{d}I$ [k$\Omega$]')
cb.set_ticks(cbar_ticks)
fig.savefig('figures/fig2.pdf', format='pdf', dpi=300)
plt.show()

# Fig 3

In [ ]:
fig, axs = plt.subplots(5, 1, sharey=False, sharex=True, figsize=(fig_width, fig_height*1.6))
plt.subplots_adjust(bottom=0.2, left=0.125, right=0.80, top=0.9, hspace=0.150, wspace=0.10)


exp = [open_mtx('data/experimental_data/fig3_1.mtx'),
       open_mtx('data/experimental_data/fig3_2.mtx'),
       open_mtx('data/experimental_data/fig3_3.mtx'),
       open_mtx('data/experimental_data/fig3_4.mtx'),
       open_mtx('data/experimental_data/fig3_5.mtx')]
    
    
fig.text(0.00, 0.6, r'$I_\textrm{bias}$ [nA]', va='center', rotation='vertical')
for i, ax in enumerate(axs.reshape(-1)):
    meta, dat = exp[i]
    meta['ymax'], meta['ymin'] = meta['ymin'], meta['ymax'] 
    if i == 0:
        ymax_new = 3.4
        ax.set_yticks([0, 1.5, 3])
    if i == 1:
        ymax_new = 2
        ax.set_yticks([0., 0.9, 1.8])
    if i == 2:
        ymax_new = 0.8
        ax.set_yticks([0, 0.3, 0.6])
    if i == 3:
        ymax_new = 0.5
        ax.set_yticks([0, 0.2, .4])
    if i == 4:
        ymax_new = 0.5
        ax.set_yticks([0, 0.2, .4])
    
    extent = (float(meta['xmin']), float(meta['xmax']), float(meta['ymin']), float(meta['ymax']))
    print(extent)
    data = np.rot90(dat) / 1000
    
    if i == 0:
        norm = HistogramNormalize(data, vmax=55, vmin=0, mixing_degree=0.5)
        kwargs = dict(extent=extent, cmap='viridis', aspect='auto', 
                      norm=norm, interpolation='nearest', origin='lower')
        im = ax.imshow(data, **kwargs)
    else:
        ax.imshow(data, **kwargs)

    ax.set_xticks([0.6, 0.7, .8])

    ax.set_ylim(meta['ymin'], ymax_new)
    ax.set_xlim(meta['xmin'], 0.88)
    ax.set_xlabel('')
#     ax.set_ylabel('$I_c$ [nA]')


    if i == 4:
        ax.set_xlabel(r'$V_\textrm{gate}$ [$V$]')
        
    ax.text(0.94, 0.72, '({})'.format('abcde'[i]),
     horizontalalignment='center', verticalalignment='center',
     transform=ax.transAxes, fontdict={'color': 'w'})

    names = [r'$0\;\mathrm{mT}$',
             r'$100\;\mathrm{mT}$',
             r'$200\;\mathrm{mT}$',
             r'$300\;\mathrm{mT}$',
             r'$400\;\mathrm{mT}$']
    ax.text(0.10, 0.72, names[i],
        horizontalalignment='left', verticalalignment='center',
        transform=ax.transAxes, fontdict={'color': 'w'})

cax = fig.add_axes([0.83, 0.2, 0.04, 0.7])
cbar_ticks = [0, 10, 20, 30, 40, 50]
cb = fig.colorbar(im, cax=cax, extend='max', label=r'$\mathrm{d}V/\mathrm{d}I$ [k$\Omega$]')
cb.set_ticks(cbar_ticks)
fig.savefig('figures/fig3.pdf', format='pdf', dpi=300, bbox_inches="tight")
plt.show()

# Fig 4

In [ ]:
from scipy.signal import argrelextrema
df = pd.read_hdf('data/current_as_function_of_B_x_with_disorder_with_combinations_of_effects.hdf')
gb = df.groupby(['orbital', 'g', 'alpha', 'mu', 'disorder', 'salt'])
total = gb.get_group((True, 50, 20, 50, 0, 0))

fig, axs = plt.subplots(4, 1, sharey=False, sharex=True, figsize=(fig_width, 1.7*fig_height), 
                        gridspec_kw={'height_ratios':[2, 1, 2, 1]})

plt.subplots_adjust(bottom=0.2, left=0.125, right=0.90, top=0.9, hspace=0.15, wspace=0.15)
current = lambda orbital=True, g=50, alpha=20, mu=10, disorder=0: (
    gb.get_group((orbital, g, alpha, mu, disorder, 0))['current_c'].values * 2.09757)
# multiplied by 2e/hbar * k_B * T
# http://www.wolframalpha.com/input/?i=(2+*+electronic+charge+%2F+hbar+)+*+boltzmann+constant+*+50e-3+kelvin

phase = lambda orbital=True, g=50, alpha=20, mu=10, disorder=0: gb.get_group((orbital, g, alpha, mu, disorder, 0))['phase_c'].values
B_x = total['B_x']

def plot_curves(ax, mu, plot=current):
    args = {'lw': 0.6, 'alpha': 1}
    ax.plot(B_x, plot(mu=mu), label='all effects', c='b', **args)
    ax.plot(B_x, plot(orbital=False, mu=mu), label=r'$\bm{A}=0$', c='g', **args)
    ax.plot(B_x, plot(g=0, mu=mu)*1.025, label='$g=0$', c='r', **args) # small off-set to make the curves not overlap
    ax.plot(B_x, plot(alpha=0, mu=mu), label=r'$\alpha=0$', c='m', **args)
    ax.plot(B_x, plot(orbital=False, alpha=0, mu=mu), label=r'$\alpha=0, \; \bm{A}=0$', c='c', **args)
    ax.plot(B_x, plot(alpha=0, g=0, mu=mu), label=r'$\alpha=0, \; g=0$', c='y', **args)
    ax.plot(B_x, plot(mu=mu, disorder=70), label=r'disorder, 70 meV', c='k', **args)
    ax.plot(B_x, plot(mu=mu, disorder=100), label=r'disorder, 100 meV', c='k', ls=':', **args)
    return ax

ymax = 0.88
axs[0] = plot_curves(axs[0], 10)
axs[0].set_yticks([0, 0.2, 0.4, 0.6])
axs[0].set_ylim(0, ymax)
axs[0].set_xlabel('')
axs[0].set_ylabel('$I_c$ [nA]')
leg0 = axs[0].legend(loc='upper center', bbox_to_anchor=(0.5, 1.85), ncol=2, prop={'size': 6})

axs[0].text(0.01, 0.82*ymax, '(a)')

ymax = 105
axs[2] = plot_curves(axs[2], 50)
axs[2].set_yticks([0, 20, 40, 60, 80])
axs[2].set_ylim(0, ymax)
axs[2].set_ylabel('$I_c$ [nA]')
axs[2].text(0.01, 0.82*ymax, '(c)')

for i in argrelextrema(current(orbital=False, alpha=0, mu=10), np.less):
    x = B_x.iloc[i]
    y1, y2 = 0, ymax
    axs[0].plot((x, x), (y1, y2), 'c--')

for i in argrelextrema(current(orbital=False, alpha=0, mu=50), np.less):
    x = B_x.iloc[i]
    y1, y2 = 0, ymax
    axs[2].plot((x, x), (y1, y2), 'c--')
    
def first_min_max(arr):
    return B_x.iloc[argrelextrema(arr, np.less)[0]], B_x.iloc[argrelextrema(arr, np.greater)[0]]

B1, B2 = first_min_max(current(orbital=False, alpha=0, mu=10)) # for next cell
B3, B4 = first_min_max(current(orbital=False, alpha=0, mu=50)) # for next cell

axs[1] = plot_curves(axs[1], 10, phase)
axs[1].set_yticks([-3.14, 0, 3.14])
axs[1].set_yticklabels(['$-\pi$', 0, '$\pi$'])
axs[1].set_ylabel(r'$\theta_c$')
axs[1].text(0.01, -2.15, '(b)')

axs[3] = plot_curves(axs[3], 50, phase)
axs[3].set_yticks([-3.14, 0, 3.14])
axs[3].set_yticklabels(['$-\pi$', 0, '$\pi$'])
axs[3].set_xlabel('$B$ [$T$]')
axs[3].set_ylabel(r'$\theta_c$')
axs[3].text(0.01, -2.1, '(d)')
fig.savefig('figures/fig4.pdf', format='pdf', dpi=300, bbox_inches="tight", additional_artists=[leg0])
plt.show()

# Fig 5

In [ ]:
fig, axs = plt.subplots(1, 2, sharey=False, sharex=True, figsize=(fig_width, fig_height))
plt.subplots_adjust(bottom=0.2, left=0.125, right=0.80, top=0.9, hspace=0.10, wspace=0.50)

df = pd.read_hdf('data/current_as_function_of_gate_and_B_x_with_disorder_small_range.hdf')
gb = df.groupby(['disorder', 'gate_size'])
gr = gb.get_group((100, 640))

extent = (gr.min()['B_x'], gr.max()['B_x'], gr.min()['V'], gr.max()['V'])
data = gr.pivot(index='B_x', columns='V', values='current_c').T * 2.09757
exp_data = np.loadtxt('data/experimental_data/fig5.dat') * 1e9
norm = HistogramNormalize(np.array(data), mixing_degree=0.5) 
kwargs = dict(extent=extent, cmap='viridis', aspect='auto', 
              norm=norm, origin='lower',)# interpolation='nearest')

im = axs[1].imshow(data, **kwargs)


kwargs['extent'] = (0, 0.420, 0.5750, 1.0750)
# kwargs['norm'] = norm = HistogramNormalize(np.array(exp_data), mixing_degree=0.5, vmax=3.2) 
im0 = axs[0].imshow(exp_data.T, **kwargs)


# multiplied by 2e/hbar * k_B * T
# http://www.wolframalpha.com/input/?i=(2+*+electronic+charge+%2F+hbar+)+*+boltzmann+constant+*+50e-3+kelvin

axs[0].set_xlabel('$B$ [$T$]')
axs[0].set_ylabel(r'$V_\textrm{gate}$ [V]')
axs[1].set_ylabel(r'$\mu$ [meV]')
axs[0].set_ylim((0.6, 1))
axs[0].set_yticks([0.6, 0.7, 0.8, 0.9, 1])
axs[1].set_xlabel('$B$ [$T$]')
axs[1].set_xticks([0, 0.2, 0.4])
yticks = [16, 18, 20, 22, 24]
# axs[1].set_yticks(yticks)
# axs[1].set_yticklabels([50 - x for x in yticks])

cax = fig.add_axes([0.83, 0.2, 0.04, 0.70])
cbar_ticks = [0, 1, 2, 3, 4, 5]
cb = fig.colorbar(im, cax=cax, 
#                   extend='max',
                  label=r'$I_c$ [nA]')
cb.set_ticks(cbar_ticks)
fig.savefig('figures/fig5.pdf', format='pdf', dpi=300, bbox_inches="tight")

In [ ]:
fig, axs = plt.subplots(2, 1, sharey=False, sharex=True, figsize=(fig_width, fig_height*1.2))
plt.subplots_adjust(bottom=0.2, left=0.1, right=0.80, top=0.9, hspace=0.10, wspace=0.50)

df = pd.read_hdf('data/current_as_function_of_gate_and_B_x_with_disorder_small_range.hdf')
gb = df.groupby(['disorder', 'gate_size'])
gr = gb.get_group((100, 640))

# multiplied by 2e/hbar * k_B * T
# http://www.wolframalpha.com/input/?i=(2+*+electronic+charge+%2F+hbar+)+*+boltzmann+constant+*+50e-3+kelvin
data = gr.pivot(index='B_x', columns='V', values='current_c').T * 2.09757
exp_data = np.loadtxt('data/experimental_data/fig5.dat') * 1e9
kwargs = dict(cmap='viridis', aspect='auto', origin='lower',)# interpolation='nearest')


extent0 = (0, 0.420, 0.5750, 1.0750)
norm0 = HistogramNormalize(np.array(exp_data), mixing_degree=0.5, vmax=3.2) 
im0 = axs[0].imshow(exp_data.T, extent=extent0, norm=norm0, **kwargs)
axs[0].set_ylabel(r'$V_\textrm{gate}$ [V]')
axs[0].set_ylim((0.6, 1))
axs[0].set_xlim((0, 0.42))
axs[0].set_yticks([0.6, 0.7, 0.8, 0.9, 1])
cax0 = fig.add_axes([0.83, 0.565, 0.04, 0.335])
cbar_ticks = [0, 1, 2, 3, 4, 5]
cb0 = fig.colorbar(im0, cax=cax0, label=r'$I_\textrm{bias}$ [nA]')
cb0.set_ticks(cbar_ticks)

extent1 = (gr.min()['B_x'], gr.max()['B_x'], gr.min()['V'], gr.max()['V'])
norm1 = HistogramNormalize(np.array(data), mixing_degree=0.5) 
im1 = axs[1].imshow(data, extent=extent1, norm=norm1, **kwargs)
axs[1].set_xlabel('$B$ [$T$]')
axs[1].set_ylabel(r'$\mu$ [meV]')
axs[1].set_xticks([0, 0.1, 0.2, 0.3, 0.4])
yticks = [16, 18, 20, 22, 24]
axs[1].set_yticks(yticks)
axs[1].set_yticklabels([50 - x for x in yticks])
cax1 = fig.add_axes([0.83, 0.20, 0.04, 0.335])
cb1 = fig.colorbar(im1, cax=cax1, label=r'$I_c$ [nA]')
cb1.set_ticks(cbar_ticks)

for i, ax in enumerate(axs):    
    ax.text(0.03, 0.85, '({})'.format('ab'[i]),
        horizontalalignment='left', verticalalignment='center',
        transform=ax.transAxes, fontdict={'color': 'w'})
fig.savefig('figures/fig5.pdf', format='pdf', dpi=300, bbox_inches="tight")